# Process the data

* Download the previous 12 months of trip data.
* Upload to GoogleCloud and create a bucket.
* Combine the data in one table.

In [ ]:
CREATE TABLE IF NOT EXISTS cyclistic.combined_trips
AS
SELECT *
FROM
  `cyclistic.*`

These are the original variables and data types.
* ride_id	 STRING
* rideable_type	STRING
* started_at	TIMESTAMP
* ended_at	TIMESTAMP
* start_station_name  STRING
* start_station_id	STRING
* end_station_name	STRING
* end_station_id	STRING
* start_lat	FLOAT
* start_lng	FLOAT
* end_lat	FLOAT
* end_lng	FLOAT
* member_casual	STRING

Count the rows. 

In [ ]:
SELECT COUNT(*)
  FROM `midyear-castle-416616.cyclistic.combined_trips`;

-- There are 5,719,877 rows.

Now, we need to calculate the number of rows with nulls.

In [ ]:
SELECT COUNT(*) - COUNT(ride_id) AS ride_id,
  COUNT(*) - COUNT(rideable_type) AS rideable_type,
  COUNT(*) - COUNT(started_at) AS started_at,
  COUNT(*) - COUNT(ended_at) AS ended_at,
  COUNT(*) - COUNT(start_station_name) AS start_station_name,
  COUNT(*) - COUNT(start_station_id) AS start_station_id,
  COUNT(*) - COUNT(end_station_name) AS end_station_name,
  COUNT(*) - COUNT(end_station_id) AS end_station_id,
  COUNT(*) - COUNT(start_lat) AS start_lat,
  COUNT(*) - COUNT(start_lng) AS start_lng,
  COUNT(*) - COUNT(end_lat) AS end_lat,
  COUNT(*) - COUNT(end_lng) AS end_lng,
  COUNT(*) - COUNT(member_casual) AS member_casual,
FROM `cyclistic.combined_trips`

Check if all rows are distinct using ride_id.
Based on the ride_id, all rows are distinct.

In [ ]:
SELECT COUNT (ride_id) - COUNT(DISTINCT ride_id)
  AS duplicate_rows
FROM `midyear-castle-416616.cyclistic.combined_trips`

Find the number of ride durations longer than 1 day (1440 minutes). 

In [ ]:
SELECT COUNT (*)
FROM `midyear-castle-416616.cyclistic.combined_trips`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 1440

-- Result: 6417

Find the number of ride durations less than 1 minute.

In [ ]:
SELECT COUNT (*)
FROM `midyear-castle-416616.cyclistic.combined_trips`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) <= 1;

-- Result: 263293

Number of rows where start_station_name or start_station_id is missing.

In [ ]:
SELECT COUNT(ride_id) AS start_station_null 
FROM `cyclistic.combined_trips`
WHERE start_station_name IS NULL OR start_station_id IS NULL;

-- Result: 875848

Number of rows where start_lat or start_lng is missing.

In [ ]:
SELECT COUNT(ride_id) AS start_loc_null 
FROM `cyclistic.combined_trips`
WHERE 
  start_lat IS NULL OR 
  start_lng IS NULL;

-- Result: 0

Number of rows where end_station_name or end_station_id is missing.

In [ ]:
SELECT COUNT(ride_id) AS end_station_null 
FROM `cyclistic.combined_trips`
WHERE end_station_name IS NULL OR end_station_id IS NULL;

-- Result: 929343

Number of rows where end_lat or end_lng is missing.

In [ ]:
SELECT COUNT(ride_id) AS end_loc_null
FROM `cyclistic.combined_trips`
WHERE
  end_lat IS NULL OR
  end_lng IS NULL;

  -- Result: 6990

Finding the number of member types.

In [ ]:
SELECT DISTINCT member_casual, COUNT(member_casual)
FROM `cyclistic.combined_trips`

GROUP BY member_casual

-- Casual Riders: 2059179
-- Member Riders: 3660698

# Clean the data

Select and delete rows where started_at, start_station_id, start_station_name, start_lat, start_lng, ended_at, end_station_id, end_station_name, end_lat, or end_lng IS NULL. Removed 1,388,170 rows.

In [ ]:
SELECT *
FROM cyclistic.combined_trips
WHERE
started_at IS NULL
OR start_station_id IS NULL
OR start_station_name IS NULL
OR start_lat IS NULL
OR start_lng IS NULL
OR ended_at IS NULL
OR end_station_id IS NULL
OR end_station_name IS NULL
OR end_lat IS NULL
OR end_lng IS NULL;

DELETE 
FROM cyclistic.combined_trips
WHERE
started_at IS NULL
OR start_station_id IS NULL
OR start_station_name IS NULL
OR start_lat IS NULL
OR start_lng IS NULL
OR ended_at IS NULL
OR end_station_id IS NULL
OR end_station_name IS NULL
OR end_lat IS NULL
OR end_lng IS NULL;

Add column for ride_length.

In [ ]:
ALTER TABLE `cyclistic.combined_trips`
ADD COLUMN ride_length int

Calculate ride lengths.

In [ ]:
UPDATE `cyclistic.combined_trips`
SET ride_length = DATETIME_DIFF(ended_at, started_at, minute)
WHERE ride_id IS NOT NULL

Find ride_length that is less than 0 minute or greater than or equal to 1440 minutes (one day).

In [ ]:
SELECT COUNT(ride_id)
FROM `cyclistic.combined_trips`
WHERE ride_length < 0 or ride_length >= 1440

Delete rows where ride_length is less than 1 minute or greater than 1440 minutes.

In [ ]:
DELETE
FROM `cyclistic.combined_trips`
WHERE ride_length < 0 or ride_length >= 1440

Add column for the month.

In [ ]:
ALTER TABLE `cyclistic.combined_trips`
ADD COLUMN month STRING;

UPDATE `cyclistic.combined_trips`
SET month = FORMAT_DATETIME("%b", DATETIME(started_at))
WHERE ride_id IS NOT NULL

Add column for day of the week.

In [ ]:
ALTER TABLE `cyclistic.combined_trips`
ADD COLUMN day_of_week STRING;

UPDATE `cyclistic.combined_trips`
SET day_of_week = FORMAT_DATETIME("%a", DATETIME(started_at))
WHERE ride_id IS NOT NULL

Count the number of remaining rows after cleaning.

In [ ]:
SELECT COUNT(DISTINCT ride_id)
FROM `cyclistic.combined_trips`

-- Result: 4331540